Import needed stuff

In [22]:
import requests
import sys
import pandas as pd
import numpy as np
from scipy import sparse
from io import StringIO
# Stupid funny thing
def progressBar(current, total, barLength = 20):
    percent = float(current) * 100 / total
    arrow   = '-' * int(percent/100 * barLength - 1) + '>'
    spaces  = ' ' * (barLength - len(arrow))
    sys.stdout.write('\rProgress: [%s%s] %d %%' % (arrow, spaces, percent))

Downloading data by http request

In [2]:
reply = requests.get('https://data.europa.eu/euodp/data/storage/f/2020-08-11T140550/Finalised%20mobilities%20started%20in%202018%20-%20KA1.csv')

Transfer in pandas

In [28]:
csv_data = reply.text.lstrip('ï»¿')
df = pd.read_csv(StringIO(csv_data), sep=';',low_memory=False)
for i, name in enumerate(df.columns):
    print(i, name)

0 Project Reference
1 Academic Year
2 Mobility Start Month
3 Mobility End Month
4 Mobility Duration
5 Activity (mob)
6 Field of Education
7 Participant Nationality
8 Education Level
9 Participant Gender
10 Participant Profile
11 Special Needs
12 Fewer Opportunities
13 Participant Age
14 Sending Country Code
15 Sending City
16 Sending Organization
17 Receiving Country Code
18 Receiving City
19 Receiving Organization
20 Participants


## Mobility network
Simply focused on the number of the excange

In [56]:
mobility_df = df.loc[:, df.columns[np.array((16, 19, 20))]]
nodes = np.array(list(set.union(set(mobility_df.loc[:, mobility_df.columns[0]]), set(mobility_df.loc[:, mobility_df.columns[1]]))))
# I don't want it to take forever
node_position = {}
for i, name in enumerate(nodes):
    node_position[name] = i
# Adjacency matrix
adj_matrix_crs = sparse.csr_matrix((nodes.size, nodes.size),dtype=int)
adj_matrix_csc = sparse.csc_matrix((nodes.size, nodes.size),dtype=int)

l = len(df.index)-1
for index, row in mobility_df.iterrows():
    adj_matrix_crs[node_position[row['Receiving Organization']],node_position[row['Sending Organization']]] += row['Participants']
    adj_matrix_csc[node_position[row['Receiving Organization']],node_position[row['Sending Organization']]] += row['Participants']
    if(index%1000==0):
        progressBar(index, l)
progressBar(l, l)
sparse.save_npz('adj_crs', adj_matrix_crs)
sparse.save_npz('adj_csc', adj_matrix_csc)


Progress: [------------------->] 100 %

In [55]:
adj_matrix_crs= sparse.load_npz('adj_crs.npz')
adj_matrix_csc= sparse.load_npz('adj_csc.npz')


In [57]:
for i in range(1000):
    progressBar(i, 999)

Progress: [>                   ] 0 %

KeyboardInterrupt: 